# Filament Benchmark

In this notebook we will benchmark a real-world biological model from a paper entitled [Magnetic dipole with a flexible tail as a self-propelling microdevice](https://doi.org/10.1103/PhysRevE.85.041502). This is a system of PDEs representing a Kirchhoff model of an elastic rod, where the equations of motion are given by the Rouse approximation with free boundary conditions.

## Model Implementation

First we will show the full model implementation. It is not necessary to understand the full model specification in order to understand the benchmark results, but it's all contained here for completeness. The model is highly optimized, with all internal vectors pre-cached, loops unrolled for efficiency (along with `@simd` annotations), a pre-defined Jacobian, matrix multiplications are all in-place, etc. Thus this model is a good stand-in for other optimized PDE solving cases.

The model is thus defined as follows:

In [19]:
using OrdinaryDiffEq, ODEInterfaceDiffEq, Sundials, DiffEqDevTools, LSODA
using Plots
gr()

Plots.GRBackend()

In [2]:
const T = Float64
abstract type AbstractFilamentCache end
abstract type AbstractMagneticForce end
abstract type AbstractInextensibilityCache end
abstract type AbstractSolver end
abstract type AbstractSolverCache end

In [3]:
struct FerromagneticContinuous <: AbstractMagneticForce
    ω :: T
    F :: Vector{T}
end

mutable struct FilamentCache{
        MagneticForce        <: AbstractMagneticForce,
        InextensibilityCache <: AbstractInextensibilityCache,
        SolverCache          <: AbstractSolverCache
            } <: AbstractFilamentCache
    N  :: Int
    μ  :: T
    Cm :: T
    x  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    y  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    z  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    A  :: Matrix{T}
    P  :: InextensibilityCache
    F  :: MagneticForce
    Sc :: SolverCache
end

In [4]:
struct NoHydroProjectionCache <: AbstractInextensibilityCache
    J         :: Matrix{T}
    P         :: Matrix{T}
    J_JT      :: Matrix{T}
    J_JT_LDLT :: Base.LinAlg.LDLt{T, SymTridiagonal{T}}
    P0        :: Matrix{T}

    NoHydroProjectionCache(N::Int) = new(
        zeros(N, 3*(N+1)),          # J
        zeros(3*(N+1), 3*(N+1)),    # P
        zeros(N,N),                 # J_JT
        Base.LinAlg.LDLt{T,SymTridiagonal{T}}(SymTridiagonal(zeros(N), zeros(N-1))),
        zeros(N, 3*(N+1))
    )
end

In [5]:
struct DiffEqSolverCache <: AbstractSolverCache
    S1 :: Vector{T}
    S2 :: Vector{T}

    DiffEqSolverCache(N::Integer) = new(zeros(T,3*(N+1)), zeros(T,3*(N+1)))
end

In [6]:
function FilamentCache(N=20; Cm=32, ω=200, Solver=SolverDiffEq)
    InextensibilityCache = NoHydroProjectionCache
    SolverCache = DiffEqSolverCache
    tmp = zeros(3*(N+1))
    FilamentCache{FerromagneticContinuous, InextensibilityCache, SolverCache}(
        N, N+1, Cm, view(tmp,1:3:3*(N+1)), view(tmp,2:3:3*(N+1)), view(tmp,3:3:3*(N+1)),
        zeros(3*(N+1), 3*(N+1)), # A
        InextensibilityCache(N), # P
        FerromagneticContinuous(ω, zeros(3*(N+1))),
        SolverCache(N)
    )
end

FilamentCache

In [7]:
function stiffness_matrix!(f::AbstractFilamentCache)
    N, μ, A = f.N, f.μ, f.A
    A[:] = eye(3*(N+1))
    for i in 1 : 3
        A[i,i] =    1
        A[i,3+i] = -2
        A[i,6+i] =  1

        A[3+i,i]   = -2
        A[3+i,3+i] =  5
        A[3+i,6+i] = -4
        A[3+i,9+i] =  1

        A[3*(N-1)+i,3*(N-3)+i] =  1
        A[3*(N-1)+i,3*(N-2)+i] = -4
        A[3*(N-1)+i,3*(N-1)+i] =  5
        A[3*(N-1)+i,3*N+i]     = -2

        A[3*N+i,3*(N-2)+i]     =  1
        A[3*N+i,3*(N-1)+i]     = -2
        A[3*N+i,3*N+i]         =  1

        for j in 2 : N-2
            A[3*j+i,3*j+i]     =  6
            A[3*j+i,3*(j-1)+i] = -4
            A[3*j+i,3*(j+1)+i] = -4
            A[3*j+i,3*(j-2)+i] =  1
            A[3*j+i,3*(j+2)+i] =  1
        end
    end
    scale!(A, -μ^4)
    nothing
end

stiffness_matrix! (generic function with 1 method)

In [8]:
function update_separate_coordinates!(f::AbstractFilamentCache, r)
    N, x, y, z = f.N, f.x, f.y, f.z
    @inbounds for i in 1 : length(x)
        x[i] = r[3*i-2]
        y[i] = r[3*i-1]
        z[i] = r[3*i]
    end
    nothing
end

function update_united_coordinates!(f::AbstractFilamentCache, r)
    N, x, y, z = f.N, f.x, f.y, f.z
    @inbounds for i in 1 : length(x)
        r[3*i-2] = x[i]
        r[3*i-1] = y[i]
        r[3*i]   = z[i]
    end
    nothing
end

function update_united_coordinates(f::AbstractFilamentCache)
    r = zeros(T, 3*length(f.x))
    update_united_coordinates!(f, r)
    r
end

update_united_coordinates (generic function with 1 method)

In [9]:
function initialize!(initial_conf_type::Symbol, f::AbstractFilamentCache)
    N, x, y, z = f.N, f.x, f.y, f.z
    if initial_conf_type == :StraightX
        x[:] = linspace(0, 1, N+1)
        y[:] = 0 .* x
        z[:] = 0 .* x
    else
        error("Unknown initial configuration requested.")
    end
    update_united_coordinates(f)
end

initialize! (generic function with 1 method)

In [10]:
function magnetic_force!(::FerromagneticContinuous, f::AbstractFilamentCache, t)
    # TODO: generalize this for different magnetic fields as well
    N, μ, Cm, ω, F = f.N, f.μ, f.Cm, f.F.ω, f.F.F
    F[1]         = -μ * Cm * cos(ω*t)
    F[2]         = -μ * Cm * sin(ω*t)
    F[3*(N+1)-2] =  μ * Cm * cos(ω*t)
    F[3*(N+1)-1] =  μ * Cm * sin(ω*t)
    nothing
end

magnetic_force! (generic function with 1 method)

In [11]:
struct SolverDiffEq <: AbstractSolver end

function (f::FilamentCache)(dr, r, p, t)
    @views f.x, f.y, f.z = r[1:3:end], r[2:3:end], r[3:3:end]
    jacobian!(f)
    projection!(f)
    magnetic_force!(f.F, f, t)
    A, P, F, S1, S2 = f.A, f.P.P, f.F.F, f.Sc.S1, f.Sc.S2

    # implement dr = P * (A*r + F) in an optimized way to avoid temporaries
    A_mul_B!(S1, A, r)
    S1 .+= F
    A_mul_B!(S2, P, S1)
    copy!(dr, S2)
    return dr
end

function (f::FilamentCache)(::Type{Val{:jac}}, J, r, p, t)
    A_mul_B!(J, f.P.P, f.A)
    nothing
end

In [12]:
function jacobian!(f::FilamentCache)
    N, x, y, z, J = f.N, f.x, f.y, f.z, f.P.J
    @inbounds for i in 1 : N
        J[i, 3*i-2]     = -2 * (x[i+1]-x[i])
        J[i, 3*i-1]     = -2 * (y[i+1]-y[i])
        J[i, 3*i]       = -2 * (z[i+1]-z[i])
        J[i, 3*(i+1)-2] =  2 * (x[i+1]-x[i])
        J[i, 3*(i+1)-1] =  2 * (y[i+1]-y[i])
        J[i, 3*(i+1)]   =  2 * (z[i+1]-z[i])
    end
    nothing
end

jacobian! (generic function with 1 method)

In [13]:
function projection!(f::FilamentCache)
    # implement P[:] = I - J'/(J*J')*J in an optimized way to avoid temporaries
    J, P, J_JT, J_JT_LDLT, P0 = f.P.J, f.P.P, f.P.J_JT, f.P.J_JT_LDLT, f.P.P0
    A_mul_Bt!(J_JT, J, J)
    LDLt_inplace!(J_JT_LDLT, J_JT)
    A_ldiv_B!(P0, J_JT_LDLT, J)
    At_mul_B!(P, P0, J)
    subtract_from_identity!(P)
    nothing
end

projection! (generic function with 1 method)

In [14]:
function subtract_from_identity!(A)
    scale!(-1, A)
    @inbounds for i in 1 : size(A,1)
        A[i,i] += 1
    end
    nothing
end

subtract_from_identity! (generic function with 1 method)

In [15]:
function LDLt_inplace!{T<:Real}(L::Base.LinAlg.LDLt{T,SymTridiagonal{T}}, A::Matrix{T})
    n = size(A,1)
    dv, ev = L.data.dv, L.data.ev
    @inbounds for (i,d) in enumerate(diagind(A))
        dv[i] = A[d]
    end
    @inbounds for (i,d) in enumerate(diagind(A,-1))
        ev[i] = A[d]
    end
    @inbounds @simd for i in 1 : n-1
        ev[i]   /= dv[i]
        dv[i+1] -= abs2(ev[i]) * dv[i]
    end
    L
end

LDLt_inplace! (generic function with 1 method)

# Investigating the model

Let's take a look at what results of the model look like:

In [16]:
function run(::SolverDiffEq; N=20, Cm=32, ω=200, time_end=1., solver=TRBDF2(autodiff=false), reltol=1e-6, abstol=1e-6)
    f = FilamentCache(N, Solver=SolverDiffEq, Cm=Cm, ω=ω)
    r0 = initialize!(:StraightX, f)
    stiffness_matrix!(f)
    prob = ODEProblem(f, r0, (0., time_end))
    sol = solve(prob, solver, dense=false, reltol=reltol, abstol=abstol)
end

run (generic function with 1 method)

This method runs the model with the `TRBDF2` method and the default parameters.

In [17]:
sol = run(SolverDiffEq())
plot(sol,vars = (0,25))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 t 
 
<polyline clip-path="url(#clip0502)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 38.8117,360.065 38.8121,360.065 38.8136,360.065 38.8156,360.065 38.8202,360.065 38.826,360.065 38.8354,360.065 38.8469,360.065 38.8626,360.065 38.8804,360.065 
 38.9022,360.065 38.9323,360.065 38.9575,360.065 38.9927,360.065 39.0267,360.065 39.0821,360.065 39.1195,360.065 39.1747,360.065 39.2425,360.063 39.2909,360.06 
 39.3551,360.053 39.4314,360.038 39.4856,360.021 39.5556,359.99 39.6368,359.938 39.6952,359.887 39.7691,359.804 39.8541,359.678 39.916,359.563 39.9934,359.387 
 40.0814,359.141 40.1467,358.922 40.2274,358.606 40.3182,358.183 40.3871,357.812 40.471,357.296 40.5647,356.633 40.6371,356.054 40.7243,355.274 40.8209,354.302 
 40.897,353.451 40.9875,352.339 41.0869,350.985 41.1667,349.796 41.2685,348.141 41.3605,346.512 41.449,344.82 41.5595,342.532 41.6417,340.706 41.7447,338.26 
 41.8492,335.599 41.9413,333.102 42.0601,329.676 42.1471,327.016 42.2511,323.672 42.3552,320.147 42.452,316.707 42.57,312.309 42.6604,308.79 42.7467,305.312 
 42.8572,300.682 42.9486,296.718 43.0626,291.593 43.1295,288.503 43.2458,282.98 43.3434,278.202 43.4507,272.808 43.5624,267.041 43.6511,262.358 43.72,258.658 
 43.8179,253.319 43.9156,247.888 44.0256,241.672 44.1108,236.783 44.1816,232.677 44.2799,226.91 44.381,220.918 44.4926,214.224 44.5826,208.785 44.6548,204.388 
 44.7575,198.107 44.8562,192.043 44.9656,185.295 45.0303,181.297 45.1467,174.105 45.2449,168.031 45.3596,160.961 45.4714,154.094 45.5683,148.169 45.6377,143.947 
 45.7414,137.684 45.8354,132.06 45.9374,126.015 45.997,122.514 46.1012,116.465 46.2021,110.695 46.3066,104.814 46.4143,98.8717 46.4956,94.4689 46.5579,91.1479 
 46.6451,86.5745 46.7502,81.1891 46.8527,76.0829 46.9486,71.4361 47.0277,67.7032 47.1372,62.7011 47.2339,58.4452 47.3416,53.8875 47.4198,50.7077 47.4813,48.2804 
 47.5665,45.0322 47.6705,41.2473 47.7697,37.8319 47.8689,34.6055 47.9556,31.9507 48.0659,28.7979 48.164,26.2066 48.2742,23.5435 48.3586,21.6849 48.4228,20.3748 
 48.5136,18.6741 48.6093,17.082 48.7087,15.6473 48.7958,14.5746 48.8712,13.7888 48.9713,12.9512 49.0654,12.3781 49.1698,11.9851 49.2592,11.8543 49.3344,11.8887 
 49.4363,12.1447 49.5296,12.593 49.6338,13.334 49.7166,14.1004 49.7841,14.8454 49.877,16.0477 49.9746,17.5242 50.077,19.3073 50.1683,21.0994 50.2444,22.7349 
 50.3485,25.1792 50.4447,27.6457 50.551,30.6016 50.6324,33.0258 50.6966,35.0349 50.7857,37.9676 50.8879,41.5271 50.9898,45.2833 51.0838,48.9242 51.1606,52.022 
 51.2676,56.5104 51.3651,60.7788 51.4719,65.6396 51.5501,69.3147 51.6111,72.2518 51.6953,76.4016 51.8029,81.858 51.9034,87.0992 52.0046,92.5188 52.0879,97.0809 
 52.2019,103.454 52.3007,109.094 52.392,114.403 52.5069,121.196 52.6077,127.249 52.6963,132.641 52.8129,139.816 52.9107,145.905 52.9944,151.159 53.1093,158.421 
 53.2046,164.487 53.2879,169.805 53.4013,177.076 53.4984,183.308 53.5852,188.893 53.6999,196.253 53.7986,202.577 53.8863,208.181 54.0019,215.53 54.1,221.72 
 54.1856,227.087 54.3006,234.228 54.397,240.154 54.4818,245.307 54.5958,252.136 54.6925,257.847 54.7793,262.891 54.8929,269.374 54.9909,274.843 55.0797,279.701 
 55.1935,285.784 55.2921,290.912 55.3816,295.442 55.495,301.015 55.5936,305.692 55.6835,309.819 55.7963,314.807 55.895,318.988 55.9863,322.699 56.0981,327.029 
 56.1976,330.68 56.2918,333.957 56.4028,337.588 56.504,340.682 56.6132,343.775 56.6947,345.92 56.755,347.412 56.8398,349.372 56.9387,351.455 57.0355,353.28 
 57.1277,354.817 57.2087,356.008 57.3132,357.32 57.4087,358.297 57.5151,359.134 57.6038,359.63 57.6754,359.893 57.7745,360.052 57.8668,359.991 57.9688,359.687 
 58.0507,359.262 58.1191,358.787 58.2126,357.963 58.3101,356.886 58.4123,355.518 58.5053,354.065 58.5835,352.69 58.689,35

The model quickly falls into a highly oscillatory mode which then dominates throughout the rest of the solution.

# Work-Precision Diagrams

Now let's build the problem and solve it once at high accuracy to get a reference solution:

In [18]:
N=20
f = FilamentCache(N, Solver=SolverDiffEq)
r0 = initialize!(:StraightX, f)
stiffness_matrix!(f)
prob = ODEProblem(f, r0, (0., 0.01))

sol = solve(prob, Vern9(), reltol=1e-14, abstol=1e-14)
test_sol = TestSolution(sol);

## High Tolerance (Low Accuracy)

### Endpoint Error

In [22]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => BS3()),
    Dict(:alg => Tsit5()),
    #Dict(:alg => ImplicitEuler(autodiff=false)),
    Dict(:alg => Trapezoid(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    Dict(:alg => rodas()),
    Dict(:alg => dop853()),
    Dict(:alg => lsoda()),
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "BS3",
    "Tsit5",
    #"ImplicitEuler",
    "Trapezoid",
    "TRBDF2",
    "radau",
    "rodas",
    "dop853",
    "lsoda"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 10 
 
 
 - 
 
 
 1.0 
 
 
 10 
 
 
 - 
 
 
 0.5 
 
 
 10 
 
 
 0.0 
 
 
 10 
 
 
 0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 BS3 
 
 
 
 Tsit5 
 
 
 
 Trapezoid 
 
 
 
 TRBDF2 
 
 
 
 radau 
 
 
 
 rodas 
 
 
 
 dop853 
 
 
 
 lsoda

In [20]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => ImplicitEuler(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false))
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "ImplicitEuler",
    "TRBDF2",
    "radau",
    "KenCarp3",
    "KenCarp4",
    "Kvaerno3",
    "Kvaerno4"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-8 
 
 
 10^-6 
 
 
 10^-4 
 
 
 10^-2 
 
 
 10^-3 
 
 
 10^-2 
 
 
 10^-1 
 
 
 10^0 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 ImplicitEuler 
 
 
 
 TRBDF2 
 
 
 
 radau 
 
 
 
 KenCarp3 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno3 
 
 
 
 Kvaerno4

### Timeseries Error

In [23]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => Trapezoid(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    Dict(:alg => rodas()),
    Dict(:alg => lsoda())
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "Trapezoid",
    "TRBDF2",
    "radau",
    "rodas",
    "lsoda"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, dense = false,
                      maxiters=Int(1e6), verbose = false, error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 10 
 
 
 - 
 
 
 1.0 
 
 
 10 
 
 
 - 
 
 
 0.5 
 
 
 10 
 
 
 0.0 
 
 
 10 
 
 
 0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 Trapezoid 
 
 
 
 TRBDF2 
 
 
 
 radau 
 
 
 
 rodas 
 
 
 
 lsoda

By looking at the previous plots, we can see that `TRBDF2` is the quicker one and `lsoda` is the slower one (the colors are similar!)

In [23]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false))
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "TRBDF2",
    "radau",
    "KenCarp3",
    "KenCarp4",
    "Kvaerno3",
    "Kvaerno4"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-8 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-4 
 
 
 10^-3 
 
 
 10^-2 
 
 
 10^-2.0 
 
 
 10^-1.5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 10^0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 TRBDF2 
 
 
 
 radau 
 
 
 
 KenCarp3 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno3 
 
 
 
 Kvaerno4

### Dense Error

In [37]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => Trapezoid(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "Trapezoid",
    "TRBDF2"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, dense=true,
                      maxiters=Int(1e6), verbose = false, dense_errors = true, error_estimate=:L2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-8 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-4 
 
 
 10^-3 
 
 
 10^-2 
 
 
 10^-2.0 
 
 
 10^-1.5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 10^0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 Trapezoid 
 
 
 
 TRBDF2

In [36]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Rosenbrock23(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false))
];

names = [
    "CVODE-BDF",
    "Rosenbrock23",
    "Rodas4",
    "TRBDF2",
    "KenCarp3",
    "KenCarp4",
    "Kvaerno3",
    "Kvaerno4"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false, dense_errors = true, error_estimate=:L2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-8 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-4 
 
 
 10^-3 
 
 
 10^-2 
 
 
 10^-2.0 
 
 
 10^-1.5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 10^0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE-BDF 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas4 
 
 
 
 TRBDF2 
 
 
 
 KenCarp3 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno3 
 
 
 
 Kvaerno4

## Low Tolerance (High Accuracy)

In [21]:
abstols=1./10.^(6:12)
reltols=1./10.^(6:12)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Vern7()),
    Dict(:alg => Vern9()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    Dict(:alg => dop853())
];

names = [
    "CVODE_BDF",
    "Vern7",
    "Vern9",
    "TRBDF2",
    "radau",
    "dop853"
];

wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-12.5 
 
 
 10^-10.0 
 
 
 10^-7.5 
 
 
 10^-5.0 
 
 
 10^-2.5 
 
 
 10^-1.5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE_BDF 
 
 
 
 Vern7 
 
 
 
 Vern9 
 
 
 
 TRBDF2 
 
 
 
 radau 
 
 
 
 dop853

In [24]:
abstols=1./10.^(6:12)
reltols=1./10.^(6:12)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => radau()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno5(autodiff=false)),
    Dict(:alg => KenCarp5(autodiff=false)),
    Dict(:alg => lsoda())
];

names = [
    "CVODE_BDF",
    "radau",
    "TRBDF2",
    "Kvaerno3",
    "KenCarp3",
    "Kvaerno4",
    "KenCarp4",
    "Kvaerno5",
    "KenCarp5",
    "lsoda"
];

wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                                    maxiters=Int(1e6), verbose = false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 11 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 9 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 10 
 
 
 - 
 
 
 1.0 
 
 
 10 
 
 
 - 
 
 
 0.5 
 
 
 10 
 
 
 0.0 
 
 
 10 
 
 
 0.5 
 
 
 10 
 
 
 1.0 
 
 
 10 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE_BDF 
 
 
 
 radau 
 
 
 
 TRBDF2 
 
 
 
 Kvaerno3 
 
 
 
 KenCarp3 
 
 
 
 Kvaerno4 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno5 
 
 
 
 KenCarp5 
 
 
 
 lsoda

By looking at the previous plots, we can see that `KenCarp3` is the quicker one and `lsoda` is the slower one (the colors are similar!)

### Timeseries Error

In [25]:
abstols=1./10.^(6:12)
reltols=1./10.^(6:12)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => radau()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno5(autodiff=false)),
    Dict(:alg => KenCarp5(autodiff=false)),
    Dict(:alg => lsoda())
];

names = [
    "CVODE_BDF",
    "radau",
    "TRBDF2",
    "Kvaerno3",
    "KenCarp3",
    "Kvaerno4",
    "KenCarp4",
    "Kvaerno5",
    "KenCarp5",
    "lsoda"
];

wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false, error_estimate = :l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 11 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 9 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 10 
 
 
 - 
 
 
 1.0 
 
 
 10 
 
 
 - 
 
 
 0.5 
 
 
 10 
 
 
 0.0 
 
 
 10 
 
 
 0.5 
 
 
 10 
 
 
 1.0 
 
 
 10 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE_BDF 
 
 
 
 radau 
 
 
 
 TRBDF2 
 
 
 
 Kvaerno3 
 
 
 
 KenCarp3 
 
 
 
 Kvaerno4 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno5 
 
 
 
 KenCarp5 
 
 
 
 lsoda

### Dense Error

In [35]:
abstols=1./10.^(6:12)
reltols=1./10.^(6:12)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => Kvaerno3(autodiff=false)),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false)),
    Dict(:alg => KenCarp4(autodiff=false)),
    Dict(:alg => Kvaerno5(autodiff=false)),
    Dict(:alg => KenCarp5(autodiff=false)),
];

names = [
    "CVODE_BDF",
    "TRBDF2",
    "Kvaerno3",
    "KenCarp3",
    "Kvaerno4",
    "KenCarp4",
    "Kvaerno5",
    "KenCarp5"
];

wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = false, dense_errors=true, error_estimate = :L2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-10 
 
 
 10^-9 
 
 
 10^-8 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-1.5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 10^0.5 
 
 
 10^1.0 
 
 
 10^1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE_BDF 
 
 
 
 TRBDF2 
 
 
 
 Kvaerno3 
 
 
 
 KenCarp3 
 
 
 
 Kvaerno4 
 
 
 
 KenCarp4 
 
 
 
 Kvaerno5 
 
 
 
 KenCarp5

# No Jacobian Work-Precision Diagrams

In the previous cases the analytical Jacobian is given and is used by the solvers. Now we will solve the same problem without the analytical Jacobian.

Note that the pre-caching means that the model is not compatible with autodifferentiation by ForwardDiff. Thus all of the native Julia solvers are set to `autodiff=false` to use DiffEqDiffTools.jl's numerical differentiation backend. We'll only benchmark the methods that did well before.

In [23]:
N=20
f = FilamentCache(N, Solver=SolverDiffEq)
r0 = initialize!(:StraightX, f)
stiffness_matrix!(f)
prob = ODEProblem((t,r,dr)-> f(t,r,dr), r0, (0., 0.01))

sol = solve(prob, Vern9(), reltol=1e-14, abstol=1e-14)
test_sol = TestSolution(sol.t, sol.u);

## High Tolerance (Low Accuracy)

In [26]:
abstols=1./10.^(3:8)
reltols=1./10.^(3:8)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => rodas()),
    #Dict(:alg => radau()),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false))
];

names = [
    "CVODE BDF",
    "TRBDF2",
    "rodas",
    #"radau",
    "KenCarp3",
    "Kvaerno4"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = true,numruns = 10)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-4 
 
 
 10^-3 
 
 
 10^-1.25 
 
 
 10^-1.00 
 
 
 10^-0.75 
 
 
 10^-0.50 
 
 
 10^-0.25 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE BDF 
 
 
 
 TRBDF2 
 
 
 
 rodas 
 
 
 
 KenCarp3 
 
 
 
 Kvaerno4

It looks like `radau` fails on this problem with low accuracy if the Jacobian is not passed, so its values should be ignored since it exits early.

## Low Tolerance (High Accuracy)

In [27]:
abstols=1./10.^(6:12)
reltols=1./10.^(6:12)
setups = [
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => rodas()),
    Dict(:alg => radau()),
    Dict(:alg => KenCarp3(autodiff=false)),
    Dict(:alg => Kvaerno4(autodiff=false))
];

names = [
    "CVODE BDF",
    "TRBDF2",
    "rodas",
    "radau",
    "KenCarp3",
    "Kvaerno4"
];


wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, 
                      maxiters=Int(1e6), verbose = true,numruns = 10)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10^-10 
 
 
 10^-9 
 
 
 10^-8 
 
 
 10^-7 
 
 
 10^-6 
 
 
 10^-5 
 
 
 10^-1.0 
 
 
 10^-0.5 
 
 
 10^0.0 
 
 
 10^0.5 
 
 
 10^1.0 
 
 
 10^1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CVODE BDF 
 
 
 
 TRBDF2 
 
 
 
 rodas 
 
 
 
 radau 
 
 
 
 KenCarp3 
 
 
 
 Kvaerno4

## Conclusion

Sundials' `CVODE_BDF` does the best in this test. When the Jacobian is given, the ESDIRK methods `TRBDF2` and `KenCarp3` are able to do almost as well as it until `<1e-6` error is needed. When Jacobians are not given, Sundials is the fastest without competition.